In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype
import scipy.stats as stats

### clean data from PsychoPy and ouptut trial-level task data

In [2]:
raw_path = '/Users/owenfriend/Documents/temple_local/prisma_TASK/data/'
raw = pd.read_csv(raw_path + '10_1_novel_scenes.csv')

In [3]:
# user start and 5 from scanner
scan_start = raw['five_received'][2]
user_start = raw['seven_received'][1]

In [4]:
clean = (raw.iloc[2:, [24, 25, 26, 27, 28, 33, 34, 35, 46, 47]]).dropna()
clean = clean.reset_index().iloc[:, 1:]
clean.insert(0, 'duration', 0, True)
clean.insert(0, 'block', 0, True)
clean = clean.rename(columns = {'block_type': 'category'})
clean = clean[['onset', 'duration', 'offset', 'trial', 'category', 'block_num', 'response', 'correct', 'image', 'session', 'day']]

In [5]:
for index, row in clean.iterrows():
    # adjust times to be relative to 5 from scanner, easier to do here than in PsychoPy
    clean.at[index, 'onset'] -= scan_start
    clean.at[index, 'offset'] -= scan_start
    
    clean.at[index, 'duration'] = clean['offset'][index] - clean['onset'][index]
    clean.at[index, 'trial'] +=1
    
    clean.at[index, 'image'] = clean['image'][index].replace('.jpg', '')
    


In [7]:
run = clean.session.values[0]
day = clean.day.values[0]
clean.to_csv(f'/Users/owenfriend/Documents/temple_local/prisma_TASK/data/run{run}-day{day}_full_output.csv')

### output events file with one row for each block of either novel or repeat for univariate contrast

In [21]:
events = pd.DataFrame(columns = ['onset', 'duration', 'block_number', 'category'])
for block in clean.block_num.unique():
    block_frame = clean[clean['block_num'] == block].reset_index()
    onset = block_frame['onset'][0]
    offset = block_frame['offset'][7]
    category = block_frame['category'][0]
    duration = offset - onset
    
    events.loc[len(events)] = [onset, duration, block, category]
    
    

In [26]:
task = 'novelty'
sub = '1'

events.to_csv(f'/Users/owenfriend/Documents/temple_local/prisma_TASK/events_files/sub-{sub}_task-{task}_run-{run}_events.tsv', sep="\t")
    